# Modèle 1 : Models Comparaison

## Librairies à installer

In [7]:
# Pour Camembert - Large
!pip install sentencepiece protobuf
import sentencepiece
import os

## Fonctions 
(prises dans Model1_implementation_sentence_transformers.ipynb)

In [3]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np

def load_and_merge_data(csv_file = '../data/solutions.csv'):
    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)
    # Renommer les colonnes
    df.columns = ['id_solution', 'categorie', 'texte']
    # Filtrer les lignes pour les catégories spécifiées
    categories_specifiees = [1, 2, 5, 6]
    df_filtre = df[df['categorie'].isin(categories_specifiees)]
    # Pivoter les données pour obtenir les colonnes 'titre', 'definition', 'application' et 'bilan énergie'
    df_pivot = df_filtre.pivot(index='id_solution', columns='categorie', values='texte').reset_index()
    # Renommer les colonnes
    df_pivot.columns = ['id_solution', 'titre', 'definition', 'application', 'bilan_energie']

    # Ajouter un point à la fin des colonnes si nécessaire
    colonnes = ['titre', 'definition', 'application', 'bilan_energie']
    for col in colonnes:
        df_pivot[col] = df_pivot[col].apply(lambda x: x.strip() + '.' if isinstance(x, str) and x.strip()[-1] != '.' else x.strip() if isinstance(x, str) else x)
    
    # Gérer les valeurs NaN lors de la fusion des colonnes
    def combine_text(row):
        text_parts = [row[col] for col in colonnes if pd.notnull(row[col])]
        return ' '.join(text_parts)
    
    # Appliquer la fonction pour créer la nouvelle colonne 'text'
    df_pivot['text'] = df_pivot.apply(combine_text, axis=1)

    # Sélectionner uniquement les colonnes 'id_solution' et 'text'
    df_final = df_pivot[['id_solution', 'text']]
    
    return df_final

# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

def pre_processing(texte):
    # Nettoyer HTML Tags
    texte = BeautifulSoup(texte, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    texte = re.sub(r'&nbsp;\.', '', texte)

    # Accents
    texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

    # Retirer les numéros
    texte = re.sub(r'\b\d+\b', '', texte)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(texte)
    phrases_propres = []
    for phrase in doc.sents:
        tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
        phrase_propre = ' '.join(tokens)
        if phrase_propre:
            phrases_propres.append(phrase_propre + ".")  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(phrases_propres)

    return cleaned_text

def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

def genere_embedding(data, output_file, quantize=False, precision="binary"):

    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_average_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_average_embedding)

    
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings
    }
    
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("Les embeddings ont été storer avec succès.")

def find_solution(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)
    
    # Convertir la liste de tableaux numpy en un seul tableau numpy
    embeddings_array = np.stack(df_embeddings['text_embedding'].values)

    # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
    similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1), embeddings_array)
    
    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = similarities.flatten()
    
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    
    return solution_list

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 6.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Données utilisées pour la comparaison

In [10]:
# Malheureusement nous n'avons pas reçu le domaine d'activité correspondant aux 
# requetes avec Kerdos.Et pour certaines requetes nous n'avons pas l'Id_solution non
# plus, dans ce cas il est remplacé par -1.
dataset_test_kerdos = [
    ["Id_solution", "Domaine_activite", "Description"],
    [724, "","C'est quoi la HP flottante ?"],
    [914, "", "Je voudrais dimensionner un panneau solaire."],
    [719, "", "Quel gain pour un variateur de vitesse ?"],
    [-1, "", "J'aimerais avoir une régulation optimisée de mon groupe froid."],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"],
    [-1, "Pharmacie", "Quelles méthodes existent pour augmenter l'efficacité de la production des médicaments ?"],
    [-1, "Pharmacy", "What methods exist to increase the efficiency of drug production?"]
]

## Fonction de test & model_pat modifié

In [5]:
# On adapte notre fonciton pour qu'elle prenne un nom de modèle en paramètre
def model_PAT(secteur, description, model, model_name):
    # Définition du nom du fichier d'embeddings
    embeddings_file_path = model_name + "_embeddings.pkl"

    # Vérifier si le fichier d'embeddings existe
    if not os.path.exists(embeddings_file_path):
        print("Le fichier d'embeddings n'existe pas. Génération des embeddings en cours...")
        # Affachage des données chargées
        df_solutions = load_and_merge_data()
        # Appliquons le traitement à la colonne text de notre df_solutions
        df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing)
        # Générer les embeddings et les sauvegarder dans le fichier
        genere_embedding(df_solutions, embeddings_file_path)
        print("Les embeddings ont été générés et sauvegardés avec succès.")

    # On commence par concaténer notre secteur et notre description.
    text = secteur + ". " + description

    # Ensuite on applique notre pré-processing
    clean_text = pre_processing(text)

    # Ensuite on cherche nos similarités 
    solutions = find_solution(clean_text, embeddings_file_path)

    # On return une liste contenant uniquement le numéros des solutions
    id_solutions = []
    for solution in solutions :
        id_solutions.append(solution[0])

    return id_solutions

def test(model, model_name) :
    # On va tester sur notre dataset_test
    for i in range(1, len(dataset_test_kerdos)):
        print("--------------------------------------------")
        print("Solution attendue : ", dataset_test_kerdos[i][0])
        print(model_PAT(dataset_test_kerdos[i][1], dataset_test_kerdos[i][2], model, model_name))

## Sentence-Transformers : paraphrase-multilingual-MiniLM-L12-v2

- Langage : Multilingue
- Max Sequence Length : 128
- Dimension de l'embedding : 384
- Nombre de paramètres : Environ 71 millions
- Temps génération embedding : 1 min 59
- Embedding size : 1.7 MB

In [25]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
test(model, "paraphrase-multilingual-MiniLM-L12-v2")

--------------------------------------------
Solution attendue :  724
Le fichier d'embeddings n'existe pas. Génération des embeddings en cours...


/tmp/ipykernel_10223/3622992920.py:51: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


Les embeddings ont été storer avec succès.
Les embeddings ont été générés et sauvegardés avec succès.
[722, 54, 1070, 724, 386, 1144, 222, 335, 1533, 329]
--------------------------------------------
Solution attendue :  914
[270, 1486, 1453, 1452, 1456, 1454, 1455, 1457, 1446, 1447]
--------------------------------------------
Solution attendue :  719


/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/sentence_transformers/util.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  b = torch.tensor(b)


[1533, 1445, 1144, 1140, 1649, 1595, 1596, 346, 54, 444]
--------------------------------------------
Solution attendue :  -1
[1049, 149, 44, 803, 148, 5, 723, 3, 259, 483]
--------------------------------------------
Solution attendue :  -1
[152, 154, 1056, 157, 145, 171, 502, 1001, 165, 153]


## Sentence-Transformers : paraphrase-multilingual-mpnet-base-v2

- distiluse-base-multilingual-cased-v1: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. Supports 15 languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish.

- distiluse-base-multilingual-cased-v2: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. This version supports 50+ languages, but performs a bit weaker than the v1 model.

- paraphrase-multilingual-MiniLM-L12-v2 - Multilingual version of paraphrase-MiniLM-L12-v2, trained on parallel data for 50+ languages.

- paraphrase-multilingual-mpnet-base-v2 - Multilingual version of paraphrase-mpnet-base-v2, trained on parallel data for 50+ languages.


In [13]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
test(model, "paraphrase-multilingual-mpnet-base-v2")

--------------------------------------------
Solution attendue :  724
Le fichier d'embeddings n'existe pas. Génération des embeddings en cours...


/tmp/ipykernel_17800/3622992920.py:51: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


Les embeddings ont été storer avec succès.
Les embeddings ont été générés et sauvegardés avec succès.
[724, 102, 722, 823, 1103, 1600, 1613, 1454, 1456, 1455]
--------------------------------------------
Solution attendue :  914
[818, 270, 1486, 858, 914, 1635, 368, 1554, 1631, 1636]
--------------------------------------------
Solution attendue :  719
[1649, 1595, 1445, 1596, 1144, 719, 1533, 1592, 1140, 1653]
--------------------------------------------
Solution attendue :  -1
[1049, 483, 259, 723, 1693, 149, 4, 918, 1130, 9]
--------------------------------------------
Solution attendue :  -1
[1056, 1001, 165, 152, 1058, 209, 153, 145, 154, 1051]
--------------------------------------------
Solution attendue :  -1
[1054, 1141, 1011, 1469, 495, 275, 930, 65, 743, 482]
--------------------------------------------
Solution attendue :  -1
[1054, 930, 495, 482, 503, 1141, 1690, 1011, 876, 1007]


## Sentence-Transformers : La BSE

- Langage : Multilingue
- Max Sequence Length : 128
- Dimension de l'embedding : 768
- Nombre de paramètres : Environ 176 millions
- Temps génération embedding : 9 min 47
- Embedding size : 3.4 MB

Link : https://www.kaggle.com/models/google/labse/frameworks/tensorFlow2/variations/labse/versions/1?tfhub-redirect=true

Note : Il semble **hyper pertinent pour lui poser une question en Anglais et qu'il réponde en Français** par exemple. Dans l'idée on pourrait prendre des requetes dans n'importe quelle langue et renvoyer correctement les solutions.

In [11]:
# Charger le modèle
model = SentenceTransformer("sentence-transformers/LaBSE")
test(model, "LaBSE")

--------------------------------------------
Solution attendue :  724
[724, 1604, 1602, 835, 1461, 1597, 1605, 310, 1070, 328]
--------------------------------------------
Solution attendue :  914
[270, 818, 1455, 1457, 1453, 1558, 1454, 1456, 1632, 914]
--------------------------------------------
Solution attendue :  719
[1533, 1589, 210, 1596, 139, 1595, 79, 1653, 1592, 63]
--------------------------------------------
Solution attendue :  -1
[1049, 1693, 918, 866, 44, 483, 19, 1128, 9, 149]
--------------------------------------------
Solution attendue :  -1
[154, 1056, 332, 157, 778, 1051, 151, 29, 153, 152]
--------------------------------------------
Solution attendue :  -1
[1054, 1080, 1141, 340, 395, 3, 1007, 995, 738, 1622]
--------------------------------------------
Solution attendue :  -1
[1054, 1080, 1141, 340, 395, 1007, 738, 995, 310, 3]


## Sentence-Transformers : Camembert Base

- Langage : Français
- Max Sequence Length : 128
- Dimension de l'embedding : 768
- Nombre de paramètres : Environ 110 millions

- Temps génération embedding : 5 min 54
- Embedding size : 3.4 MB

On dev :
- Prearson correlation : 86.73
- Spearman correlation : 86.54

Link : https://huggingface.co/dangvantuan/sentence-camembert-base

In [28]:
# Charger le modèle
model = SentenceTransformer("dangvantuan/sentence-camembert-base")
test(model, "Camembert-base")

--------------------------------------------
Solution attendue :  724
Le fichier d'embeddings n'existe pas. Génération des embeddings en cours...


/tmp/ipykernel_10223/3622992920.py:51: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


Les embeddings ont été storer avec succès.
Les embeddings ont été générés et sauvegardés avec succès.
[1602, 1604, 835, 724, 834, 1609, 1610, 1603, 388, 1458]
--------------------------------------------
Solution attendue :  914
[1454, 1456, 1455, 1457, 270, 818, 73, 1452, 179, 1632]
--------------------------------------------
Solution attendue :  719
[1592, 1653, 1445, 1533, 1589, 79, 719, 1596, 1014, 1595]
--------------------------------------------
Solution attendue :  -1
[19, 1049, 922, 918, 44, 1037, 1048, 969, 3, 17]
--------------------------------------------
Solution attendue :  -1
[154, 720, 153, 36, 502, 778, 1056, 146, 390, 1051]


## Sentence-Transformers : Camembert Large

- Langage : Français
- Max Sequence Length : 128
- Dimension de l'embedding : 768
- Nombre de paramètres : Environ 340 millions
- Temps génération embedding : 14m 
- Embedding size : 4.5 MB

On dev :
- Prearson correlation : 88.2
- Spearman correlation : 88.02

Link : https://huggingface.co/dangvantuan/sentence-camembert-base

In [12]:
# Charger le modèle
model = SentenceTransformer("dangvantuan/sentence-camembert-large")
test(model, "Camembert-large")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with MEAN pooling.


--------------------------------------------
Solution attendue :  724
[1604, 835, 724, 1602, 1603, 1608, 1606, 723, 1610, 1609]
--------------------------------------------
Solution attendue :  914
[270, 1454, 1456, 1632, 1457, 1455, 1448, 1450, 914, 1452]
--------------------------------------------
Solution attendue :  719
[1596, 1595, 719, 1533, 1445, 63, 79, 1589, 1592, 1653]
--------------------------------------------
Solution attendue :  -1
[1049, 758, 1114, 19, 1048, 867, 294, 3, 34, 922]
--------------------------------------------
Solution attendue :  -1
[154, 153, 1056, 778, 152, 332, 165, 170, 150, 1092]
--------------------------------------------
Solution attendue :  -1
[495, 3, 1561, 482, 494, 974, 1497, 236, 1578, 1010]
--------------------------------------------
Solution attendue :  -1
[1561, 1010, 974, 495, 1559, 943, 427, 970, 482, 1054]


## Comparaison modèles (sur laptop)

| Nom modèle | Langage | Temps génération embedding | Taille embedding | Temps requette | Accuracy
|------------|---------|----------------------------|------------------|----------------|---------|
| paraphrase-multilingual-MiniLM-L12-v2 | Multi | 2min| 1.7MB| 0.1s| |
| paraphrase-multilingual-mpnet-base-v2 | Multi | 8min30| 3.4MB| 0.1s| |
| sentence-transformers/LaBSE | Multi | 10min| 3.4MB| 0.1s| |
| dangvantuan/sentence-camembert-base | Fr | 6min|3.4MB| 0.1s| | 
| dangvantuan/sentence-camembert-large| Fr | 14min|4.5MB| 0.1s| |


# Fine tunning

Notre dataset n'étant pas très grand et étant ammené à évolué avec le temps, nous avons décidé de ne pas fine tuner notre modèle dessus, afin de ne pas le spécialiser sur le peu de données que nous possédons en prenant le risque de perdre en généralisation sur le moyen/long terme.

# NOTES

Notre étude va se dérouler en plusieurs parties. 

1. Mise en place d'un modèle de base :

    **pre_traitement()**

    Entrée : nos csv
    
    Sortie : Une matrice contenant les données que nous pensons utile pour ce modèle après avoir enlevé les stop_words, la ponctuation et les majuscules.

    
    **Choix d'un modèle de base**

    **genere_embedding()**

    Entrée : la sortie de pre_traitement()
    
    Sortie : stock dans un fichier les embeddings correspondant à nos solutions.

    **find_solution()**

    Entrée : secteur, description

    Sortie : Une liste des meilleures solutions classés dans l'orde. On peut imaginer mettre une limite par exemple distance cos > 0.65

    **Tester conso dans gaia**

2. Creation d'un Test set 

    **Creation du test_set**

    On génère des description et domaine d'activité en fonction des infos de solution avec GPT4. On génère différent niveau de précision par solution.

    **test_model()**

    Entrée : find_solution() de notre modèle et test_set.

    Sortie : On renvoie une accuracy. On considère que si le modèle prédit la bonne solution dans son top 3 sol, c'est validé.

3. Test de différents modèles. L'idée est d'avoir un tableau avec la conso, l'accuracy et le model pour expliquer pourquoi on a pris celui-ci plutôt qu'un autre.

    **Faire un tableau de tests**
    
    Tester différents modèles et type de modèles, Word2Vec, FastText, SentenceTransformers (plusieurs).
    
    **Test Fine tuning**
    
    Fine tunner le meilleur pour voir le résultat.

    **Si temps, on peut même tenter d'entrainer un modèle sur notre test set avec notre méthode**



## 2. Génération test set 

En utilisant GPT4, prendre des lots de 20 solutions, générer doaine d'activité et description de ce qu'un utilisateur pourrait demander. En générer 3 ou 4 par solutions avec différents niveau de précision. Le faire pour une centaine de solutions.

On aurrait donc X = [description, domaine_activitée], y = [num_solution ].

Pour tester notre modèle on peut tenter de lui faire prédire les meilleures solutions en fonction de la description et du domaine_activitée. Si la solution correspondante est dans le top 3 des solutions prédites, on considère que c'est une réussite.

# 3. Modèles à tester


TF-IDF (c'est long et peu performant mais méthode sans embedding. On peut en parler dans Questionnaire et dire qu'on ne l'a pas implémenté car peu pertinent suivant les études qu'on a lu.)

FastText (on a un token par mot et on fait la moyenne pour avoir le texte)

Sentence transformers (un token par phrase et on fait la moyenne pour avoir le texte)

Doc2Vec (document lvl)

